## Introduction
TODO

In [1]:
# Run for required packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix #accuracy metrics for each classification
from sklearn.metrics import accuracy_score

## Dataset Description
Our dataset consists of quantitative descriptions of various wines. There are 1599 wines sampled. We have 11 features for each observation, each with a numeric rating of a physiochemical property of the wine such as alcohol content and acidity. The output variable we seek to predict corresponds to the quality rating of each wine on a scale from 0 to 10, from sensory data. There are no missing values. 

In [12]:
filename = "./winequality-red.csv"
df = pd.read_csv(filename, delimiter = ';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


## Experimental Setup 

We are using Python with the numpy and pandas libraries, as well as classifiers from the sklearn library. We are processing the data by (...). The algorithms we are considering are (...). For each algorithm (including choice of hyperparameters) we are using K-folds cross validation with K=5 and for each fold checking the mean accuracy. (But question - if we use a classifier but also e.g. linear regression, how do we compare performance?)

In [24]:
X = df.values[:,0:-1]
y = df.values[:,-1]
cross_val_score(RandomForestClassifier(), X, y, cv=5) # Gives the accuracy of each of K fold tests for a Random Forest with all default hyperparameters

array([0.515625  , 0.55625   , 0.590625  , 0.60625   , 0.57366771])

### Random Forest Classifier

In [ ]:
cross_val_score(RandomForestClassifier(), X, y, cv=5) # Gives the accuracy of each of K fold tests for a Random Forest 

In [14]:
#This was just me doing the above the longer way to see all the pieces
kf = KFold(n_splits = 5)
scores = np.zeros(5)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train = df.values[train_index, 0:-1]
    y_train = df.values[train_index, -1]
    X_test = df.values[test_index, 0:-1]
    y_test = df.values[test_index, -1]
    rfmodel = RandomForestClassifier().fit(X_train, y_train)
    scores[i]= rfmodel.score(X_test,y_test)